In [ ]:
#IGNORE_FOR_NOW default_exp ml.data

# Image dataset from tabular

> Create a viseme image dataset from an existing tabular dataset.

# TODO: not had time for this yet

In [1]:
#export
from expoco.core import *
from expoco.camera_capture import *
from pathlib import Path
import numpy as np
import pandas as pd
import json, cv2

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

In [2]:
from zipfile import ZipFile
COLUMN_NAMES = landmark_ids_to_col_names(FaceLandmarks.pointer, None, ['x','y'])
def image_dataset_from_capture_sessions(input_path='data/capture_session', glob_pattern='viseme*'):
    input_path, dataset_id = Path(input_path), now()
    expressions = {
        '0': "NoExpression",
        '1': "oo",
        '2': "ee",
        '3': "ah",
        '4': "RandomTalking"
    }
    path = Path(f'data/viseme_image_dataset_{dataset_id}')
    path.mkdir(parents=True, exist_ok=False)
    
    face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
    
    data = dict(image_path=[], expression_id=[], expression=[])
    for col in COLUMN_NAMES: data[col] = []
    with ZipFile(path/f'viseme_image_dataset_{dataset_id}.zip', 'w') as z, \
            ImageHelper() as image_helper:
        for session_path in sorted(input_path.glob(glob_pattern)):
            with open(session_path/'metadata.json') as f:
                session_metadata = json.load(f)
                expression_id = session_metadata['expression_id']
                expression = expressions[expression_id]
            for image_file in session_path.glob('*.jpeg'):
                image = image_helper.read_and_process(image_file)
                arcname = f'{expression}/{image_file.parent.name}_{image_file.stem}.png'
                data['image_path'].append(arcname)
                data['expression_id'].append(expression_id)
                data['expression'].append(expression)
                
                results = face_mesh.process(image_helper.cropped_rgb_image)
                for landmark_id in FaceLandmarks.pointer:
                    landmark = results.multi_face_landmarks[0].landmark[landmark_id]
                    for coord in ['x','y']:
                        data[f'{landmark_id}{coord}'].append(getattr(landmark, coord))
                
                i_png = str(path/'i.png')
                cv2.imwrite(i_png, image)
                z.write(i_png, arcname)
                
    pd.DataFrame(data).to_csv(path/'data.csv', index=False)
    Path(i_png).unlink()
    return path

In [3]:
path = image_dataset_from_capture_sessions()

In [4]:
pd.read_csv(path/'data.csv')

,image_path,expression_id,expression,1x,1y,2x,2y,5x,5y,218x,218y,438x,438y
0,NoExpression/viseme_20211115_142443_0_1.png,0,NoExpression,0.511305,0.522590,0.507947,0.555671,0.512735,0.445999,0.442406,0.514077,0.574370,0.517491
1,NoExpression/viseme_20211115_142443_0_10.png,0,NoExpression,0.489094,0.510566,0.489178,0.541003,0.491822,0.434202,0.425114,0.501078,0.553947,0.505360
2,NoExpression/viseme_20211115_142443_0_100.png,0,NoExpression,0.506577,0.506456,0.504835,0.540394,0.506581,0.431824,0.437969,0.500864,0.570789,0.501885
3,NoExpression/viseme_20211115_142443_0_101.png,0,NoExpression,0.499881,0.510537,0.500598,0.544206,0.499864,0.434750,0.435518,0.505101,0.563584,0.505261
4,NoExpression/viseme_20211115_142443_0_102.png,0,NoExpression,0.502358,0.506116,0.501110,0.540697,0.502708,0.428979,0.435199,0.499637,0.567015,0.500640
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,RandomTalking/viseme_20211116_112635_4_95.png,4,RandomTalking,0.517799,0.587163,0.512924,0.609594,0.519668,0.504506,0.444659,0.569932,0.585035,0.571577
9996,RandomTalking/viseme_20211116_112635_4_96.png,4,RandomTalking,0.523611,0.586246,0.517459,0.608266,0.525263,0.503880,0.449698,0.569144,0.589410,0.570480
9997,RandomTalking/viseme_20211116_112635_4_97.png,4,RandomTalking,0.521509,0.576739,0.516371,0.600519,0.523531,0.495488,0.449439,0.560304,0.587204,0.563178
9998,RandomTalking/viseme_20211116_112635_4_98.png,4,RandomTalking,0.528838,0.569292,0.523046,0.593147,0.529964,0.487949,0.456247,0.553183,0.593504,0.555051
